In [3]:
import pandas
from scipy.stats import norm
import math

In [4]:
walmart_df = pandas.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/walmart.csv')
zips_df = pandas.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/zips.csv')

In [5]:
def calculate_sample_size(df, confidence_interval, margin_of_error):
    # Population size
    N = len(df)
    
    # Z-score
    z_score = norm.ppf((1 + confidence_interval) / 2)
    
    # proportion
    p = 0.5
    
    # calculate sample size
    numerator = (z_score**2) * p * (1 - p)
    denominator = (margin_of_error**2) + ((z_score**2) * p * (1 - p) / N)
    sample_size = numerator / denominator
    
    return math.ceil(sample_size)

confidence_interval = 0.95
margin_of_error = 0.05

required_sample_size = calculate_sample_size(zips_df, confidence_interval, margin_of_error)
required_sample_size

380

In [8]:
# Determine the proportionate sample size for each stratum
total_zip_count = zips_df.shape[0]
sample_size = 380  # Adjust based on your requirements
zips_df['stratum_size'] = zips_df.groupby(['state_name', 'ruca'])['zip'].transform('count')
zips_df['sample_proportion'] = zips_df['stratum_size'] / total_zip_count
zips_df['stratum_sample_size'] = (zips_df['sample_proportion'] * sample_size).round().astype(int)

# Stratified sampling of ZIP codes
sampled_zips = zips_df.groupby(['state_name', 'ruca']).apply(lambda x: x.sample(n=x['stratum_sample_size'].iloc[0], random_state=1)).reset_index(drop=True)

# Random sampling of Walmart stores
sampled_walmarts = walmart_df.sample(n=sample_size, random_state=1)  # Adjust sample size as necessary

# Export sampled data
sampled_zips.to_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/interim/zips_sample.csv', index=False)
sampled_walmarts.to_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/interim/walmart_sample.csv', index=False)